# Health Economics Analysis Demo

This notebook demonstrates finbot's health economics toolkit, which adapts
the platform's Monte Carlo simulation and optimization frameworks to health
economics contexts.

## Tools Demonstrated

1. **QALY Monte Carlo Simulator** — Model health intervention outcomes with probabilistic uncertainty
2. **Cost-Effectiveness Analysis** — Compare interventions using ICER, NMB, and CEAC
3. **Treatment Schedule Optimizer** — Grid search over dose frequency and duration

## Key Concepts

- **QALY** (Quality-Adjusted Life Year): 1 year of perfect health = 1.0 QALY
- **ICER** (Incremental Cost-Effectiveness Ratio): additional cost per additional QALY
- **NMB** (Net Monetary Benefit): WTP x incremental_QALYs - incremental_cost
- **CEAC** (Cost-Effectiveness Acceptability Curve): P(cost-effective) vs WTP threshold
- **WTP** (Willingness-to-Pay): threshold for acceptable cost per QALY ($50K-$150K in US)

In [ ]:
# Setup
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['DYNACONF_ENV'] = 'development'

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from finbot.services.health_economics.qaly_simulator import (
    HealthIntervention,
    simulate_qalys,
)
from finbot.services.health_economics.cost_effectiveness import (
    cost_effectiveness_analysis,
)
from finbot.services.health_economics.treatment_optimizer import (
    optimize_treatment,
)

print('Health economics modules loaded successfully.')

## 1. QALY Monte Carlo Simulation

Simulate a hypothetical drug that improves quality of life and reduces mortality.
We model uncertainty in both cost and effectiveness using Monte Carlo draws.

In [ ]:
# Define two interventions
drug_a = HealthIntervention(
    name='Drug A',
    cost_per_year=8000.0,
    cost_std=1000.0,
    utility_gain=0.12,        # Improves utility by 0.12/year
    utility_gain_std=0.03,
    mortality_reduction=0.005, # Reduces annual mortality by 0.5%
    mortality_reduction_std=0.002,
)

no_treatment = HealthIntervention(name='No Treatment')

# Run simulations
sim_drug = simulate_qalys(
    drug_a,
    baseline_utility=0.65,
    baseline_mortality=0.03,
    time_horizon=15,
    n_sims=10_000,
    seed=42,
)

sim_none = simulate_qalys(
    no_treatment,
    baseline_utility=0.65,
    baseline_mortality=0.03,
    time_horizon=15,
    n_sims=10_000,
    seed=42,
)

print(f'Drug A  — Mean QALYs: {sim_drug["mean_qaly"]:.3f}, Mean Cost: ${sim_drug["mean_cost"]:,.0f}')
print(f'No Tx   — Mean QALYs: {sim_none["mean_qaly"]:.3f}, Mean Cost: ${sim_none["mean_cost"]:,.0f}')
print(f'Incremental QALYs: {sim_drug["mean_qaly"] - sim_none["mean_qaly"]:.3f}')
print(f'Incremental Cost:  ${sim_drug["mean_cost"] - sim_none["mean_cost"]:,.0f}')

In [ ]:
# Visualize QALY distributions
fig = make_subplots(rows=1, cols=2, subplot_titles=['Total QALYs', 'Total Cost'])

fig.add_trace(go.Histogram(x=sim_drug['total_qalys'], name='Drug A', opacity=0.7), row=1, col=1)
fig.add_trace(go.Histogram(x=sim_none['total_qalys'], name='No Treatment', opacity=0.7), row=1, col=1)
fig.add_trace(go.Histogram(x=sim_drug['total_costs'], name='Drug A', opacity=0.7, showlegend=False), row=1, col=2)

fig.update_layout(title='QALY and Cost Distributions (Drug A vs No Treatment)', template='plotly_white', barmode='overlay')
fig.update_xaxes(title_text='QALYs', row=1, col=1)
fig.update_xaxes(title_text='Cost ($)', row=1, col=2)
fig.show()

In [ ]:
# Survival curves comparison
surv_drug = sim_drug['survival_curves'].mean()
surv_none = sim_none['survival_curves'].mean()

fig = go.Figure()
fig.add_trace(go.Scatter(x=list(surv_drug.index), y=surv_drug.values, mode='lines+markers', name='Drug A'))
fig.add_trace(go.Scatter(x=list(surv_none.index), y=surv_none.values, mode='lines+markers', name='No Treatment'))
fig.update_layout(
    title='Mean Survival Curves',
    xaxis_title='Year', yaxis_title='Survival Probability',
    template='plotly_white', yaxis_range=[0, 1],
)
fig.show()

## 2. Cost-Effectiveness Analysis

Compare Drug A against No Treatment using standard health economics metrics:
ICER, NMB, CEAC, and the cost-effectiveness plane.

In [ ]:
# Run CEA
cea = cost_effectiveness_analysis(
    sim_results={'Drug A': sim_drug, 'No Treatment': sim_none},
    comparator='No Treatment',
)

# ICER results
print('Cost-Effectiveness Summary')
print('=' * 60)
print(cea['summary'].to_string())
print()
print(f'ICER: ${cea["icer"]["ICER"].iloc[0]:,.0f} per QALY')
print()
icer_val = cea['icer']['ICER'].iloc[0]
if icer_val < 50_000:
    print('Interpretation: Highly cost-effective (ICER < $50,000/QALY)')
elif icer_val < 100_000:
    print('Interpretation: Cost-effective (ICER < $100,000/QALY)')
elif icer_val < 150_000:
    print('Interpretation: Marginally cost-effective ($100K-$150K/QALY)')
else:
    print('Interpretation: Not cost-effective at standard WTP thresholds')

In [ ]:
# Cost-effectiveness plane
plane = cea['ce_plane']['Drug A']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=plane['Delta QALYs'], y=plane['Delta Cost'],
    mode='markers', marker=dict(size=2, opacity=0.2, color='blue'),
    name='Simulations',
))

# Add WTP threshold line ($50K/QALY)
x_range = [plane['Delta QALYs'].min(), plane['Delta QALYs'].max()]
fig.add_trace(go.Scatter(
    x=x_range, y=[x * 50_000 for x in x_range],
    mode='lines', line=dict(dash='dash', color='red'),
    name='WTP $50K/QALY',
))
fig.add_trace(go.Scatter(
    x=x_range, y=[x * 100_000 for x in x_range],
    mode='lines', line=dict(dash='dash', color='orange'),
    name='WTP $100K/QALY',
))

fig.add_hline(y=0, line_dash='dot', line_color='gray')
fig.add_vline(x=0, line_dash='dot', line_color='gray')
fig.update_layout(
    title='Cost-Effectiveness Plane (Drug A vs No Treatment)',
    xaxis_title='Incremental QALYs', yaxis_title='Incremental Cost ($)',
    template='plotly_white',
)
fig.show()

# Quadrant analysis
ne = ((plane['Delta QALYs'] > 0) & (plane['Delta Cost'] > 0)).mean() * 100
se = ((plane['Delta QALYs'] > 0) & (plane['Delta Cost'] <= 0)).mean() * 100
nw = ((plane['Delta QALYs'] <= 0) & (plane['Delta Cost'] > 0)).mean() * 100
sw = ((plane['Delta QALYs'] <= 0) & (plane['Delta Cost'] <= 0)).mean() * 100
print(f'Quadrant distribution:')
print(f'  NE (more effective, more costly): {ne:.1f}%')
print(f'  SE (dominant — more effective, less costly): {se:.1f}%')
print(f'  NW (dominated — less effective, more costly): {nw:.1f}%')
print(f'  SW (less effective, less costly): {sw:.1f}%')

In [ ]:
# CEAC and NMB
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Cost-Effectiveness Acceptability Curve', 'Net Monetary Benefit'],
)

ceac = cea['ceac']
for col in ceac.columns:
    fig.add_trace(go.Scatter(x=ceac.index, y=ceac[col], mode='lines', name=col), row=1, col=1)

nmb = cea['nmb']
for col in nmb.columns:
    fig.add_trace(go.Scatter(x=nmb.index, y=nmb[col], mode='lines', name=col, showlegend=False), row=1, col=2)
fig.add_hline(y=0, line_dash='dash', line_color='red', row=1, col=2)

fig.update_xaxes(title_text='WTP Threshold ($/QALY)', row=1, col=1)
fig.update_xaxes(title_text='WTP Threshold ($/QALY)', row=1, col=2)
fig.update_yaxes(title_text='P(Cost-Effective)', range=[0, 1], row=1, col=1)
fig.update_yaxes(title_text='NMB ($)', row=1, col=2)
fig.update_layout(template='plotly_white', title='Decision Analysis')
fig.show()

# Find WTP where drug becomes preferred
ceac_drug = ceac['Drug A']
crossover = ceac_drug[ceac_drug > 0.5].index
if len(crossover) > 0:
    print(f'Drug A becomes preferred (>50% probability) at WTP >= ${crossover[0]:,.0f}/QALY')

## 3. Treatment Schedule Optimization

Find the optimal dose frequency and treatment duration to maximize
Net Monetary Benefit (NMB). This adapts the DCA optimizer pattern
from financial portfolio optimization.

In [ ]:
# Optimize treatment schedule
opt_results = optimize_treatment(
    cost_per_dose=800.0,
    cost_per_dose_std=100.0,
    qaly_gain_per_dose=0.015,
    qaly_gain_per_dose_std=0.003,
    frequencies=[1, 2, 4, 12, 26, 52],
    durations=[1, 2, 3, 5, 10, 15],
    baseline_utility=0.65,
    baseline_mortality=0.03,
    wtp_threshold=100_000.0,
    n_sims=5000,
    seed=42,
)

print('Top 5 Treatment Schedules by NMB:')
print('=' * 80)
top5 = opt_results.head()
for _, row in top5.iterrows():
    print(f'  {int(row["Frequency"]):>2} doses/yr x {int(row["Duration"]):>2} yr  |  '
          f'ICER: ${row["ICER"]:>10,.0f}/QALY  |  NMB: ${row["NMB"]:>10,.0f}')

best = opt_results.iloc[0]
print(f'\nOptimal: {int(best["Frequency"])} doses/yr for {int(best["Duration"])} years')
print(f'  Annual cost: ${best["Annual_Cost"]:,.0f}')
print(f'  Total cost:  ${best["Total_Cost"]:,.0f}')
print(f'  QALYs gained: {best["Incremental_QALYs"]:.3f}')
print(f'  ICER: ${best["ICER"]:,.0f}/QALY')
print(f'  NMB:  ${best["NMB"]:,.0f}')

In [ ]:
# Visualize optimization results
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['NMB by Schedule', 'ICER by Schedule'],
)

for dur in sorted(opt_results['Duration'].unique()):
    sub = opt_results[opt_results['Duration'] == dur]
    fig.add_trace(go.Scatter(
        x=sub['Frequency'], y=sub['NMB'],
        mode='lines+markers', name=f'{int(dur)}yr',
    ), row=1, col=1)
    fig.add_trace(go.Scatter(
        x=sub['Frequency'], y=sub['ICER'].clip(upper=500_000),
        mode='lines+markers', name=f'{int(dur)}yr', showlegend=False,
    ), row=1, col=2)

fig.add_hline(y=0, line_dash='dash', line_color='red', row=1, col=1)
fig.add_hline(y=100_000, line_dash='dash', line_color='green', row=1, col=2)

fig.update_xaxes(title_text='Doses per Year', row=1, col=1)
fig.update_xaxes(title_text='Doses per Year', row=1, col=2)
fig.update_yaxes(title_text='NMB ($)', row=1, col=1)
fig.update_yaxes(title_text='ICER ($/QALY)', row=1, col=2)
fig.update_layout(template='plotly_white', title='Treatment Schedule Optimization')
fig.show()

## Key Findings

1. **QALY Simulation**: Monte Carlo simulation captures the full uncertainty in health
   outcomes, producing distributions rather than point estimates for both costs and QALYs.

2. **Cost-Effectiveness Plane**: The scatter plot reveals the joint uncertainty in
   incremental costs and QALYs. Points in the SE quadrant (dominant) are ideal; NE
   quadrant points depend on the WTP threshold.

3. **CEAC**: Shows how the probability of cost-effectiveness changes with the WTP
   threshold, helping decision-makers understand sensitivity to their valuation of health.

4. **Treatment Optimization**: Grid search over dose frequency and duration reveals
   that the optimal schedule balances treatment intensity against cost, with diminishing
   returns at very high frequencies.

5. **Framework Reuse**: The same Monte Carlo and optimization patterns used for
   financial analysis (portfolio simulation, DCA optimization) translate directly to
   health economics — demonstrating the analytical framework's versatility.

## Next Steps

- Compare multiple drugs simultaneously (multi-arm CEA)
- Add Bayesian value of information analysis (EVPI, EVPPI)
- Model treatment switching and dynamic protocols
- Incorporate real-world effectiveness data from clinical trials
- Budget impact analysis for health system planning